In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchfly.modules.transformers import CachedBertDecoderLM, ChineseBERTBaseConfig
from torchfly.text.tokenizers import BertTokenizer
from torchfly.utils import get_pretrained_states

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

100%|██████████| 109540/109540 [00:00<00:00, 455666.13B/s]


In [3]:
model_states = get_pretrained_states("chinese-gpt-bert-small")

Cached Downloading: /home/wuqy1203/.cache/torchfly/models/chinese-gpt-bert-small.pth
Downloading...
From: https://drive.google.com/uc?id=1agi64d06PlBe6XUz2IMkgl8ZKjw6H7nS
To: /home/wuqy1203/.cache/gdown/tmp5brfz297/dl
407MB [00:17, 22.6MB/s] 


In [4]:
model = CachedBertDecoderLM(ChineseBERTBaseConfig)
model.load_state_dict(model_states, strict=False)

<All keys matched successfully>

In [5]:
device = torch.device("cuda")
model = model.to(device)

In [6]:
def top_k_logits(logits, k):
    """Mask logits so that only top-k logits remain
    """
    values, _ = torch.topk(logits, k)
    min_values = values[:, -1].unsqueeze(1).repeat(1, logits.shape[-1])
    return torch.where(logits < min_values, torch.ones_like(logits, dtype=logits.dtype) * -1e10, logits)

In [7]:
prompt = tokenizer.encode("阿里巴巴集团宣布收购雅虎")
batch_size = 1

In [8]:
top_k = 50
temperature = 0.8
length = 0

start_predictions = torch.LongTensor([[101] + prompt]* batch_size).to(device)
mask = torch.ones(batch_size, start_predictions.shape[1]).to(device)

past = None

with torch.no_grad():
    # cache saves in past
    logits, past = model(start_predictions, mask, past=None, past_length=0)
    logits = logits[:, -1, :] / temperature
    logits = top_k_logits(logits, k=top_k)

    sentence = []

    probs = F.softmax(logits, dim=-1)
    prob, prev_pred = torch.topk(probs, k=1, dim=-1)
    sentence.append(prev_pred)
    length += 1

    # decoding loop
    for i in range(500):
        mask = F.pad(mask, (0, 1), "constant", 1.0)
        logits, past = model(prev_pred, mask, past=past, past_length=length)
        logits = logits.squeeze(1) / temperature
        logits = top_k_logits(logits, k=top_k)
        probs = F.softmax(logits, dim=-1)
        prev_pred = torch.multinomial(probs, num_samples=1)
        sentence.append(prev_pred)
        length += 1

    sentence = torch.cat(sentence, dim=-1)



In [9]:
tokenizer.decode(sentence[0].tolist())

'的创认为，这次交时间炸机是因为他本身的产品经验，3/4的股权是因为他提供了多个合作伙伴，当时，雅虎联合创始人员员工在阿里大战中的表现就不容易，这在全球上是不多的了，也从不能令人信赖。在这样一个巨大的市场中，大型公司，雅虎也有自己独特的优势，在网络上不可避免的会员人数占比的不利，这样的人员配备和使用方便是他们在网上最大的优势。公司创始人们还是不忘经营的人，阿里旗下的几家互联网公司也是不会放弃任何一个公司的。1．利用搜索引擎的特点：这个引擎在搜索引擎中的作用不仅仅在搜索结果中，还会为了增加网站内容数量，使用这个引擎时，我们将会得到一些相关的建立工具，这些网站内容在网站文章中一目了然，而且在网站外包给中国用的时候，就是因为我们能够够了。2．雅虎搜索引擎的分类是一项很实用的工具，他的含义与前几代的"原型"和"合作"有显著区别。3．雅虎的网络推广应用的方法简单、便利，不仅仅是以搜索引擎的名义进行的。4．使用雅虎"出生的第一天子"属于什么类型？---雅虎"出生的第一天子"就是第三个时期5．什么是"出生的第一天子"，这是两个字母的组合，而且是不是意在改变?如果是不能6．雅虎是个人，是一种时间，或者是一'